##### Copyright 2022 The Cirq Developers

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello Qubit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/start/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
# Import necessary libraries for elliptic curve operations
try:
    from fastecdsa.point import Point
    from fastecdsa.curve import secp256k1
except ImportError:
    print("installing fastecdsa...")
    !pip install --quiet fastecdsa
    from fastecdsa.point import Point
    from fastecdsa.curve import secp256k1
    print("installed fastecdsa.")

import threading
import time

# === CONFIG ===
TARGET_PUBKEY_HEX = "04a9c9c642db7941be7d9d467786c05b66d13c4fdafeb81d66815fb336e7e19d42981f62f2624826898152444b99b590476c90ea7f2e15f50a00e726368d6ec7c1"
TARGET_PUBKEY_BYTES = bytes.fromhex(TARGET_PUBKEY_HEX)
TARGET_PUBKEY_X = int.from_bytes(TARGET_PUBKEY_BYTES[1:33], 'big')
TARGET_PUBKEY_Y = int.from_bytes(TARGET_PUBKEY_BYTES[33:], 'big')
TARGET_PUBKEY_POINT = Point(TARGET_PUBKEY_X, TARGET_PUBKEY_Y, curve=secp256k1)

# Standard Generator Point for secp256k1
G = secp256k1.G

# === Brute-Force Function ===
def classical_brute_force_range(start_priv_key, end_priv_key, target_pubkey):
    print(f"Thread searching range: {start_priv_key} to {end_priv_key}")
    for priv_key in range(start_priv_key, end_priv_key + 1):
        # Perform scalar multiplication
        candidate_pubkey = G * priv_key

        # Check if the candidate public key matches the target
        if candidate_pubkey == target_pubkey:
            print(f"\n✅ SUCCESS: Private key found: {priv_key}")
            print(f"Private key (hex): {priv_key:064x}")
            return priv_key # Found it!

    print(f"Thread finished searching range: {start_priv_key} to {end_priv_key} - Not found.")
    return None # Not found in this range

# === Parallel Search (Classical) ===
def parallel_classical_search(start_range, end_range, num_threads=4):
    threads = []
    total_keys_in_range = end_range - start_range + 1
    keys_per_thread = total_keys_in_range // num_threads
    found_key = None

    start_time = time.time()

    for i in range(num_threads):
        thread_start = start_range + i * keys_per_thread
        thread_end = thread_start + keys_per_thread - 1
        if i == num_threads - 1: # Ensure the last thread covers the end of the range
            thread_end = end_range

        thread = threading.Thread(target=classical_brute_force_range,
                                  args=(thread_start, thread_end, TARGET_PUBKEY_POINT))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join() # Wait for all threads to complete

    end_time = time.time()
    print(f"\nSearch finished in {end_time - start_time:.2f} seconds.")

# === Run the search ===
# Define your desired search range here
# Example: Searching a range of 2^33 keys starting from 0
SEARCH_RANGE_START = 2**33
SEARCH_RANGE_END = 2**55

parallel_classical_search(SEARCH_RANGE_START, SEARCH_RANGE_END)

installing fastecdsa...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.6 MB/s eta 0:00:00
installed fastecdsa.
Thread searching range: 8589934592 to 9007205697191935
Thread searching range: 9007205697191936 to 18014402804449279
Thread searching range: 18014402804449280 to 27021599911706623
Thread searching range: 27021599911706624 to 36028797018963968


# Congratulations
You've just run your first Cirq program.

To learn about running a circuit on a virtual machine that mimics existing quantum hardware, see [Quantum Virtual Machine](../simulate/quantum_virtual_machine.ipynb).

If you would like to learn more about quantum computing, check out our [education page](https://quantumai.google/resources). The Full API reference for Cirq can be found [here](/reference/python/cirq). If you are looking for vendor specific information that can be found on our vendor sub-pages:


  [Alpine Quantum Technologies](../hardware/aqt/getting_started.ipynb)
  
  [Pasqal](../hardware/pasqal/getting_started.ipynb)
  
  [IonQ](../hardware/ionq/getting_started.ipynb)
  
  [Azure](../hardware/azure-quantum/getting_started_honeywell.ipynb)